In [1]:
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine

Create sqlalchemy engine and read tables from DB

In [2]:
engine = create_engine('sqlite:///../data/football_new.db')

recruits = pd.read_sql_table('cfb_recruits', engine)
draftees = pd.read_sql_table('nfl_draft', engine)

Perform initial merge of recruits and draft tables using the player names

In [3]:
drafted = pd.merge(recruits, draftees, how='inner', on='PLAYER')

- Identify duplicate names in merged dataframe and create new dataframe with just the duplicates
- Remove duplciated rows where the school & college do nto match
- Drop any leftover duplicated rows

In [4]:
mask = drafted.duplicated('PLAYER', keep=False)
df_duplicates = drafted[mask]

df_duplicates = df_duplicates.drop(df_duplicates[df_duplicates['COLLEGE'] != df_duplicates['SCHOOL']].index)
df_duplicates = df_duplicates.drop_duplicates('PLAYER', keep='last')

Create create new dataframe from 'drafted' that contains no duplicated names

In [5]:
df_no_duplicates = drafted.drop_duplicates('PLAYER', keep=False)

Concatenate both dataframes to create a new 'drafted' dataframe sans duplicate rows

In [6]:
drafted = pd.concat([df_no_duplicates, df_duplicates], ignore_index=True)
drafted.head()

,PLAYER,FIRST_NAME_x,LAST_NAME_x,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,CITY,...,LNG,HIGH_SCHOOL,YEAR_y,RND,PICK,FIRST_NAME_y,LAST_NAME_y,TM,POS_y,SCHOOL
0,Joe McKnight,Joe,McKnight,RB,2007,1,5,0.9997,USC,New Orleans,...,-89.975005,John Curtis,2010,4,112,Joe,McKnight,NYJ,RB,USC
1,Tyrann Mathieu,Tyrann,Mathieu,CB,2010,223,4,0.9132,LSU,New Orleans,...,-89.975005,St. Augustine,2013,3,69,Tyrann,Mathieu,ARI,DB,LSU
2,Trai Turner,Trai,Turner,OG,2011,355,3,0.8765,LSU,New Orleans,...,-89.975005,St. Augustine,2014,3,92,Trai,Turner,CAR,OL,LSU
3,Deion Jones,Deion,Jones,OLB,2012,378,3,0.8899,LSU,New Orleans,...,-89.975005,Jesuit,2016,2,52,Deion,Jones,ATL,OLB,LSU
4,Lorenzo Doss,Lorenzo,Doss,ATH,2012,1123,3,0.8354,Tulane,New Orleans,...,-89.975005,St. Augustine,2015,5,164,Lorenzo,Doss,DEN,CB,Tulane


Confirm there are no null values

In [7]:
print(f'NaN count: {drafted.isnull().sum().sum()}')

NaN count: 0


Drop extraneous columns

In [8]:
drop_columns = ['FIRST_NAME_x','LAST_NAME_x','HIGH_SCHOOL','FIRST_NAME_y','LAST_NAME_y']

drafted = drafted.drop(drop_columns, axis=1)
drafted.head()

,PLAYER,POS_x,YEAR_x,RNK,STAR,247_SCORE,COLLEGE,CITY,STATE,LAT,LNG,YEAR_y,RND,PICK,TM,POS_y,SCHOOL
0,Joe McKnight,RB,2007,1,5,0.9997,USC,New Orleans,LA,30.032802,-89.975005,2010,4,112,NYJ,RB,USC
1,Tyrann Mathieu,CB,2010,223,4,0.9132,LSU,New Orleans,LA,30.032802,-89.975005,2013,3,69,ARI,DB,LSU
2,Trai Turner,OG,2011,355,3,0.8765,LSU,New Orleans,LA,30.032802,-89.975005,2014,3,92,CAR,OL,LSU
3,Deion Jones,OLB,2012,378,3,0.8899,LSU,New Orleans,LA,30.032802,-89.975005,2016,2,52,ATL,OLB,LSU
4,Lorenzo Doss,ATH,2012,1123,3,0.8354,Tulane,New Orleans,LA,30.032802,-89.975005,2015,5,164,DEN,CB,Tulane


Rename dataframe columns

In [10]:
drafted = drafted.rename(columns={
    'POS_x':'POS_CFB',
    'YEAR_x':'YEAR_RECRUITED',
    'RNK':'RNK_RECRUITED',
    'COLLEGE':'COLLEGE_CFB',
    'CITY':'HOME_CITY',
    'STATE':'HOME_STATE',
    'YEAR_y':'YEAR_DRAFTED',
    'RND':'DRAFT_RND',
    'PICK':'DRAFT_PICK',
    'TM':'NFL_TEAM',
    'POS_y':'POS_NFL',
    'SCHOOL':'COLLEGE_NFL'
})
drafted.head()

,PLAYER,POS_CFB,YEAR_RECRUITED,RNK_RECRUITED,STAR,247_SCORE,COLLEGE_CFB,HOME_CITY,HOME_STATE,LAT,LNG,YEAR_DRAFTED,DRAFT_RND,DRAFT_PICK,NFL_TEAM,POS_NFL,COLLEGE_NFL
0,Joe McKnight,RB,2007,1,5,0.9997,USC,New Orleans,LA,30.032802,-89.975005,2010,4,112,NYJ,RB,USC
1,Tyrann Mathieu,CB,2010,223,4,0.9132,LSU,New Orleans,LA,30.032802,-89.975005,2013,3,69,ARI,DB,LSU
2,Trai Turner,OG,2011,355,3,0.8765,LSU,New Orleans,LA,30.032802,-89.975005,2014,3,92,CAR,OL,LSU
3,Deion Jones,OLB,2012,378,3,0.8899,LSU,New Orleans,LA,30.032802,-89.975005,2016,2,52,ATL,OLB,LSU
4,Lorenzo Doss,ATH,2012,1123,3,0.8354,Tulane,New Orleans,LA,30.032802,-89.975005,2015,5,164,DEN,CB,Tulane


Export to CSV

In [12]:
drafted.to_csv('../data/drafted_recruits.csv', index=False)